<a href="https://colab.research.google.com/github/ShahedSabab/maskOrNot/blob/master/maskDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import re

# Defining Model

In [0]:
num_classes = 2
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Dense(num_classes, activation='softmax'))

# not using the first layer for training
model.layers[0].trainable = False

# Compile Model

In [0]:
model.compile(optimizer='sgd', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

# Image augmentation for preprocessing image

In [0]:
# Specify the values for all arguments to data_generator_with_aug.
data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                              horizontal_flip = True, 
                                              width_shift_range = 0.1,
                                              height_shift_range = 0.1)
            
# Speicy no augmentation that will be used for validation data
data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

# Take image directly from director and apply augmentation

In [14]:
#define image size height and width
image_size = 224

#Prepare training image
train_generator = data_generator_with_aug.flow_from_directory(
                                        directory='input/mask-detection/images/train',
                                        classes = ['with_mask', 'without_mask'],
                                        target_size=(image_size, image_size),
                                        batch_size=10,
                                        class_mode='categorical')
#Prepare validation image
validation_generator = data_generator_no_aug.flow_from_directory(
                                        directory='input/mask-detection/images/val',
                                        classes = ['with_mask', 'without_mask'],
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

Found 1074 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


# Fit the model

In [15]:
# fit_stats below saves some statistics describing how model fitting went
# the key role of the following line is how it changes my_new_model by fitting to data
fit_stats = model.fit_generator(train_generator,
                                       steps_per_epoch=60,
                                       epochs = 4,
                                       validation_data=validation_generator,
                                       validation_steps=1)

Epoch 1/4
60/60 [==============================] - 11s 175ms/step - loss: 0.1938 - accuracy: 0.9200 - val_loss: 0.0899 - val_accuracy: 0.9688
Epoch 2/4
60/60 [==============================] - 10s 164ms/step - loss: 0.0566 - accuracy: 0.9832 - val_loss: 0.0329 - val_accuracy: 1.0000
Epoch 3/4
60/60 [==============================] - 10s 159ms/step - loss: 0.0403 - accuracy: 0.9867 - val_loss: 0.0111 - val_accuracy: 1.0000
Epoch 4/4
60/60 [==============================] - 10s 160ms/step - loss: 0.0378 - accuracy: 0.9883 - val_loss: 0.0169 - val_accuracy: 1.0000


# testing with test image

In [62]:
#collect test data from directory and process
test_generator = data_generator_no_aug.flow_from_directory(
    directory='input/mask-detection/images/test',
    target_size=(image_size, image_size),
    batch_size= 10,
    class_mode=None,
    shuffle=False
)

# Predict from generator (returns probabilities)
pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
# Get classes by np.round
cl = np.round(pred)
# Get filenames (set shuffle=false in generator is important)
filenames=test_generator.filenames
# Data frame

real_class = []
for file in filenames:
  if re.search("with_mask", file):
    real_class.append(1.0)
  else:
    real_class.append(0.0)

predicted_class = cl[:,0]

results=pd.DataFrame({"file":filenames,"pr":pred[:,0], "pred_class":predicted_class, "real_class":real_class})

Found 200 images belonging to 3 classes.
20/20 [==============================] - 2s 123ms/step


In [63]:
pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
cl = np.round(pred)

20/20 [==============================] - 2s 122ms/step


In [64]:
results

,file,pr,pred_class,real_class
0,with_mask/train_00000859.jpg,0.997383,1.0,1.0
1,with_mask/train_00000860.jpg,0.998884,1.0,1.0
2,with_mask/train_00000861.jpg,1.000000,1.0,1.0
3,with_mask/train_00000862.jpg,0.999984,1.0,1.0
4,with_mask/train_00000863.jpg,1.000000,1.0,1.0
...,...,...,...,...
195,without_mask/53_0_0_20170120223141508.jpg,0.001081,0.0,0.0
196,without_mask/53_0_0_20170120223144035.jpg,0.000446,0.0,0.0
197,without_mask/5_0_3_20170119212110570.jpg,0.145459,0.0,0.0
198,without_mask/5_1_3_20170119151512160.jpg,0.067580,0.0,0.0


In [65]:
res = sum(1 for x,y in zip(real_class,predicted_class) if x == y) / len(real_class)
print("Accuracy :", res)

Accuracy : 0.99


# Save Model

In [0]:
model.save('maskDetectionModel.h5')